In [1]:
# Import Dependencies
import pandas as pd
from datetime import datetime

# Dependencies
import requests
import json

# Import the API key
from config import geoapify_key

In [2]:
# Import CSV, convert to dataframe and inspect
df = pd.read_csv('2022wildlifePatients.csv')
df.head()

,patient_id,admission_date,species,common_name,found_city,found_county,status,final_date
0,2022-0001,1/1/22,MUDU,Muscovy duck,NaN,NaN,D24,1/1/22
1,2022-0002,1/2/22,BDOW,Barred owl,Buladeen,NaN,E,1/4/22
2,2022-0003,1/5/22,EASO,Eastern screech owl,NaN,NaN,E,1/20/22
3,2022-0004,1/8/22,EASO,Eastern screech owl,Wilkesboro,Wilkes,R,2/9/22
4,2022-0005,1/9/22,HAWO,Hairy woodpecker,Blowing Rock,Watauga,D24,1/10/22


In [3]:
# Create 'AnimalNames' Table
unique_species = df['species'].unique()
unique_common_animals = df['common_name'].unique()
animal_counts = df.groupby(['species']).size().reset_index(name='species_count')

new_df = pd.DataFrame({'species': unique_species, 'common_name': unique_common_animals})
new_df = pd.merge(new_df, animal_counts, "inner")


new_df.loc[new_df.index[:90], 'animal_class'] = 'Bird'
new_df.loc[new_df.index[90:100], 'animal_class'] = 'Reptile'
new_df.loc[new_df.index[100:], 'animal_class'] = 'Mammal'

new_df

,species,common_name,species_count,animal_class
0,MUDU,Muscovy duck,2,Bird
1,BDOW,Barred owl,19,Bird
2,EASO,Eastern screech owl,41,Bird
3,HAWO,Hairy woodpecker,4,Bird
4,RTHA,Red-tailed hawk,27,Bird
...,...,...,...,...
109,RBNU,Red-breasted nuthatch,1,Mammal
110,BLCA,Southern short-tailed shrew,1,Mammal
111,LAGE,Eastern Kingsnake,1,Mammal
112,DOWO,Downy woodpecker,1,Mammal


In [4]:
# Create 'StatusCodes' Table
unique_codes = df['status'].unique()
code_counts = df.groupby(['status']).size().reset_index(name='status_count')
status_meaning = ['Died w/in 24 hrs', 'Euthanized','Released','Tamed','Died','Euth w/in 24 hrs', 'Euth on Arrival','Dead on Arrival','Rehabbed','Partial Release','Escaped']
new_df2 = pd.DataFrame({'status': unique_codes, 'status_meaning': status_meaning})
new_df2 = pd.merge(new_df2, code_counts, 'inner')
new_df2

,status,status_meaning,status_count
0,D24,Died w/in 24 hrs,165
1,E,Euthanized,112
2,R,Released,585
3,T,Tamed,42
4,D,Died,155
5,E24,Euth w/in 24 hrs,59
6,EOA,Euth on Arrival,37
7,DOA,Dead on Arrival,52
8,Reh,Rehabbed,10
9,PR,Partial Release,4


In [5]:
# Create 'Places' Table

county_counts = df.groupby(['found_county']).size().reset_index(name='county_count')

unique_counties = df['found_county'].unique()
nc_series = pd.Series(['NC'] * len(unique_counties))
code_series = pd.Series(['37'] * len(unique_counties))
new_df3 = pd.DataFrame({'found_county':unique_counties, 'state':nc_series, 'state_code':code_series})


new_df3 = pd.merge(new_df3, county_counts, "inner")


new_df3.loc[new_df3['found_county'] == 'Grayson', 'state'] = 'VA'
new_df3.loc[new_df3['found_county'] == 'Grayson', 'state_code'] = '51'
new_df3.loc[new_df3['found_county'] == 'Greenville', 'state'] = 'SC'
new_df3.loc[new_df3['found_county'] == 'Greenville', 'state_code'] = '45'

new_df3 = new_df3.drop(36)
new_df3 = new_df3.drop(31)

new_df3

,found_county,state,state_code,county_count
0,Wilkes,NC,37,36
1,Watauga,NC,37,193
2,Cabarrus,NC,37,2
3,Avery,NC,37,166
4,Buncombe,NC,37,114
5,Henderson,NC,37,17
6,Burke,NC,37,45
7,Rutherford,NC,37,5
8,Mitchell,NC,37,59
9,Macon,NC,37,2


In [6]:
# Add latitude and longitude for counties
lat = [];
lon = [];


for index, row in new_df3.iterrows():
    if row['found_county'] == 'Grayson':
        target = f"Independence, VA"
        target_url = f"https://api.geoapify.com/v1/geocode/search?text={target}&format=json&apiKey={geoapify_key}"
        geo_data = requests.get(target_url).json()
        lat.append(geo_data['results'][0]['lat'])
        lon.append(geo_data['results'][0]['lon'])
    else:
        target = f"{row['found_county']} County, {row['state']}"
        target_url = f"https://api.geoapify.com/v1/geocode/search?text={target}&format=json&apiKey={geoapify_key}"
        geo_data = requests.get(target_url).json()
        lat.append(geo_data['results'][0]['lat'])
        lon.append(geo_data['results'][0]['lon'])


new_df3['county_lat'] = lat
new_df3['county_lon'] = lon
new_df3

,found_county,state,state_code,county_count,county_lat,county_lon
0,Wilkes,NC,37,36,36.199825,-81.134135
1,Watauga,NC,37,193,36.222811,-81.686381
2,Cabarrus,NC,37,2,35.404550,-80.564371
3,Avery,NC,37,166,36.077129,-81.923052
4,Buncombe,NC,37,114,35.629222,-82.525560
5,Henderson,NC,37,17,35.338906,-82.466392
6,Burke,NC,37,45,35.727040,-81.663211
7,Rutherford,NC,37,5,35.399463,-81.902319
8,Mitchell,NC,37,59,36.000118,-82.134903
9,Macon,NC,37,2,35.143664,-83.397730


In [7]:
# Create 'Patients' Table
df.head()


df.loc[df['final_date'] == '0000-00-00', 'final_date'] = '12/31/22'

date1_str = df['admission_date']
date2_str = df['final_date']


# Create empty list to store datetime objects
date1 = []
date2 = []

# Loop over date strings and convert to datetime objects
for d1, d2 in zip(date1_str, date2_str):
    date1.append(datetime.strptime(d1, '%m/%d/%y'))
    date2.append(datetime.strptime(d2, '%m/%d/%y'))

# Calculate the number of days between the two dates for each pair
num_days = []
for d1, d2 in zip(date1, date2):
    num_days.append((d2 - d1).days)

# Print the number of days for each pair
df['stay_length'] = num_days

new_df4 = df.drop('common_name', axis=1)
new_df4

,patient_id,admission_date,species,found_city,found_county,status,final_date,stay_length
0,2022-0001,1/1/22,MUDU,NaN,NaN,D24,1/1/22,0
1,2022-0002,1/2/22,BDOW,Buladeen,NaN,E,1/4/22,2
2,2022-0003,1/5/22,EASO,NaN,NaN,E,1/20/22,15
3,2022-0004,1/8/22,EASO,Wilkesboro,Wilkes,R,2/9/22,32
4,2022-0005,1/9/22,HAWO,Blowing Rock,Watauga,D24,1/10/22,1
...,...,...,...,...,...,...,...,...
1217,2022-1220,12/27/22,YBSA,NaN,NaN,R,2/8/23,43
1218,2022-1221,12/27/22,EASO,Burnsville,Yancey,R,1/6/23,10
1219,2022-1222,12/29/22,AMRO,Morganton,Burke,R,1/9/23,11
1220,2022-1223,12/31/22,EABL,Taylor,Buncombe,R,2/8/23,39


In [8]:
# Save new dfs as tables to be uploaded to database
new_df.to_csv("Animals.csv", index=False)
new_df2.to_csv("StatusCodes.csv", index=False)
new_df3.to_csv("Places.csv", index=False)
new_df4.to_csv("Patients.csv", index=False)